In [85]:
import pandas as pd
import re
import urllib.request
import json

from selenium import webdriver
import time

from scrapy.selector import Selector
from scrapy.utils.markup import remove_tags
# from scrapy.http import HtmlResponse
from tqdm import tqdm_notebook as tqdm

from datetime import datetime
import numpy as np

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [172]:
from tinydb import TinyDB, Query
db = TinyDB('./db/db.json')

In [86]:
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

In [96]:
# TODO: переписать на селениум
# функция собирает ссылки на пивоварни со страницы с топ-50-пивоварнями выбранной страны
def get_breweries_by_country(rating_urls):
    d = {'brewery_url': [], 'country': [], 'brewery_name': []}
    for url in tqdm(rating_urls):
        opener = AppURLopener()
        response = opener.open(url)
        html = response.file.read()

        links = Selector(text=html).xpath("//*[contains(@class, 'name')]/a/@href").extract()
        countries = Selector(text=html).xpath("//*[contains(@class, 'style')]/text()").extract()
        names = Selector(text=html).xpath("//*[contains(@class, 'name')]/a/text()").extract()
        try:
            assert (len(links) == len(countries) == len(names))
        except:
            print('Different rows count for country:', url)
            break
        d['brewery_name'].extend(names)
        d['brewery_url'].extend(['https://untappd.com' + l for l in links])
        d['country'].extend(countries)
        time.sleep(1)
    return d

In [309]:
# функция возвращает перечень позиций пива и их х-к, указанных пивоварней
def get_beers_by_brewery(brewery_urls):
    d = {'url': [], 'name': [], 'style': [], 'abv': [], 'ibu': [], 'rating': [],
         'raters': [], 'added': [], 'brewery_url': [], 'brewery_added': [], 'description': []}
    
    options = webdriver.ChromeOptions()
    options.binary_location = "/Applications/Google Chrome 2.app/Contents/MacOS/Google Chrome"
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(brewery_urls[0] + '/beer')
    html = driver.page_source.encode('utf-8')
    # Click button to login screen
    time.sleep(7)
    driver.find_element_by_css_selector('.beer-list .button').click()
    # Login
    driver.find_element_by_id('username').send_keys('mershov')
    driver.find_element_by_id('password').send_keys('Aa1234567')
    driver.find_element_by_css_selector('span.button.yellow.submit-btn').click()
    
    for url in tqdm(brewery_urls):
        print(url, '...')
        url = url + '/beer'
        page_num = 0
        driver.get(url)
        
        # Click button 'Show more' while not error
        while driver.find_elements_by_css_selector('.beer-list .button'):#.beer-list .button
            try:
                driver.find_element_by_css_selector('.beer-list .button').click()#.beer-list .button
            except Exception as e:
                print(e)
                break
            page_num += 1
            print("getting page number "+str(page_num))
            if(page_num > 10):
                time.sleep(np.random.randint(25, 35))
            else:
                time.sleep(np.random.randint(7, 13))
                
        html = driver.page_source.encode('utf-8')
        
        names = Selector(text=html).xpath("//*[contains(@class, 'name')]/a/text()").extract()
        
        brewery_urls = [url] * len(names)
        
        brewery_added = Selector(text=html).xpath("//*[contains(@class, 'details brewery claimed') or contains(@class, 'details brewery')]//*[contains(@class, 'date')]/text()").extract()
        brewery_added = list(map(lambda s: re.findall(r'(\d{2}\/\d{2}\/\d{2})', s)[0], brewery_added))
        brewery_added = brewery_added * len(names)        
        
        urls = Selector(text=html).xpath("//*[contains(@class, 'name')]/a/@href").extract()
        
        styles = Selector(text=html).xpath("//*[contains(@class, 'beer-details')]//p[2]/text()").extract()
        
        abvs = Selector(text=html).xpath("//*[contains(@class, 'abv')]/text()").extract()
        abvs = list(map(lambda s: re.findall(r'\d{1,2}[.]?[\d{1,2}]?|N/A', s)[0], abvs))
        
        ibus = Selector(text=html).xpath("//*[contains(@class, 'ibu')]/text()").extract()
        ibus = list(map(lambda s: re.findall(r'(\d{1,3}|N/A) IBU', s)[0], ibus))
        
        ratings = Selector(text=html).xpath("//*[contains(@class, 'beer-item')]//*[contains(@class, 'num')]").extract()
        ratings = list(map(remove_tags, ratings))
        ratings = list(map(lambda s: re.findall(r'(\d{1}[\,\.]{0,1}\d{0,2}|N/A)', s)[0], ratings))
        
        raters = Selector(text=html).xpath("//*[contains(@class, 'beer-item')]//*[contains(@class, 'raters')]/text()").extract()
        raters = list(map(lambda s: re.findall(r'(\d{1,3}[\,\.]{0,1}\d{0,3}) Rating', s)[0], raters))
        raters = list(map(lambda s: s.replace(',', ''), raters))
        
        addeds = Selector(text=html).xpath("//*[contains(@class, 'beer-item')]//*[contains(@class, 'date')]/text()").extract()
        addeds = list(map(lambda s: re.findall(r'(\d{2}\/\d{2}\/\d{2})', s)[0], addeds))
        
        descriptions = Selector(text=html).xpath("//*[contains(@class, 'beer-details')]//p[4]").extract()
        descriptions = list(map(remove_tags, descriptions))
        descriptions = list(map(lambda s: s.replace(' Read Less', ''), descriptions))
        
        try:
            assert (len(names) == len(urls) ==\
                    len(brewery_urls) == len(descriptions) ==\
                    len(styles) == len(abvs) == len(ibus) ==\
                    len(raters) == len(ratings) == len(addeds) ==\
                    len(brewery_added))
        except:
            print('Different rows count for brewery:', url)
            continue
        d['name'].extend(names)
        d['brewery_url'].extend(brewery_urls)
        d['brewery_added'].extend(brewery_added)
        d['url'].extend(['https://untappd.com' + l for l in urls])
        d['style'].extend(styles)
        d['abv'].extend(abvs)
        d['ibu'].extend(ibus)
        d['rating'].extend(ratings)
        d['raters'].extend(raters)
        d['added'].extend(addeds)
        d['description'].extend(descriptions)
        time.sleep(np.random.randint(15, 25))
    return d

In [310]:
# list_ = '''<option value="130">Afghanistan</option>
# <option value="198">Albania</option>
# <option value="220">Algeria</option>
# <option value="176">American Samoa</option>
# <option value="224">Andorra</option>
# <option value="215">Angola</option>
# <option value="229">Antigua and Barbuda</option>
# <option value="26">Argentina</option>
# <option value="131">Armenia</option>
# <option value="27">Aruba</option>
# <option value="28">Australia</option>
# <option value="29">Austria</option>
# <option value="132">Azerbaijan</option>
# <option value="115">Bahamas</option>
# <option value="134">Bangladesh</option>
# <option value="196">Barbados</option>
# <option value="161">Belarus</option>
# <option value="30">Belgium</option>
# <option value="31">Belize</option>
# <option value="230">Benin</option>
# <option value="116">Bermuda</option>
# <option value="206">Bhutan</option>
# <option value="117">Bolivia</option>
# <option value="162">Bosnia and Herzegovina</option>
# <option value="96">Botswana</option>
# <option value="118">Brazil</option>
# <option value="245">British Virgin Islands</option>
# <option value="133">Brunei</option>
# <option value="163">Bulgaria</option>
# <option value="226">Burkina Faso</option>
# <option value="216">Burundi</option>
# <option value="135">Cambodia</option>
# <option value="97">Cameroon</option>
# <option value="32">Canada</option>
# <option value="221">Cape Verde</option>
# <option value="201">Cayman Islands</option>
# <option value="250">Central African Republic</option>
# <option value="231">Chad</option>
# <option value="218">Channel Islands</option>
# <option value="119">Chile</option>
# <option value="33">China / People's Republic of China</option>
# <option value="34">Colombia</option>
# <option value="210">Cook Islands</option>
# <option value="120">Costa Rica</option>
# <option value="35">Croatia</option>
# <option value="36">Cuba</option>
# <option value="238">Curacao</option>
# <option value="164">Cyprus</option>
# <option value="37">Czech Republic</option>
# <option value="189">Democratic Republic of the Congo</option>
# <option value="38">Denmark</option>
# <option value="209">Dominica</option>
# <option value="121">Dominican Republic</option>
# <option value="93">Ecuador</option>
# <option value="39">Egypt</option>
# <option value="40">El Salvador</option>
# <option value="41">England</option>
# <option value="98">Eritrea</option>
# <option value="42">Estonia</option>
# <option value="99">Ethiopia</option>
# <option value="252">Falkland Islands</option>
# <option value="232">Faroe Islands</option>
# <option value="177">Fiji</option>
# <option value="43">Finland</option>
# <option value="44">France</option>
# <option value="45">French Polynesia</option>
# <option value="244">Gabon</option>
# <option value="100">Gambia</option>
# <option value="137">Georgia</option>
# <option value="46">Germany</option>
# <option value="101">Ghana</option>
# <option value="165">Gibraltar</option>
# <option value="47">Greece</option>
# <option value="205">Greenland</option>
# <option value="242">Grenada</option>
# <option value="178">Guam</option>
# <option value="122">Guatemala</option>
# <option value="233">Guinea</option>
# <option value="222">Guyana</option>
# <option value="123">Haiti</option>
# <option value="49">Honduras</option>
# <option value="138">Hong Kong</option>
# <option value="50">Hungary</option>
# <option value="166">Iceland</option>
# <option value="51">India</option>
# <option value="139">Indonesia</option>
# <option value="140">Iran</option>
# <option value="141">Iraq</option>
# <option value="52">Ireland</option>
# <option value="195">Isle of Man</option>
# <option value="53">Israel</option>
# <option value="54">Italy</option>
# <option value="219">Ivory Coast</option>
# <option value="55">Jamaica</option>
# <option value="56">Japan</option>
# <option value="142">Jordan</option>
# <option value="143">Kazakhstan</option>
# <option value="57">Kenya</option>
# <option value="192">Kosovo</option>
# <option value="144">Kuwait</option>
# <option value="211">Kyrgyzstan</option>
# <option value="193">Laos</option>
# <option value="59">Latvia</option>
# <option value="145">Lebanon</option>
# <option value="236">Lesotho</option>
# <option value="208">Liberia</option>
# <option value="251">Libya</option>
# <option value="167">Liechtenstein</option>
# <option value="60">Lithuania</option>
# <option value="168">Luxembourg</option>
# <option value="146">Macau</option>
# <option value="217">Madagascar</option>
# <option value="102">Malawi</option>
# <option value="147">Malaysia</option>
# <option value="214">Mali</option>
# <option value="170">Malta</option>
# <option value="62">Mauritius</option>
# <option value="63">Mexico</option>
# <option value="179">Micronesia</option>
# <option value="171">Moldova</option>
# <option value="148">Mongolia</option>
# <option value="172">Montenegro</option>
# <option value="103">Morocco</option>
# <option value="202">Mozambique</option>
# <option value="223">Myanmar (Burma)</option>
# <option value="64">Namibia</option>
# <option value="149">Nepal</option>
# <option value="65">Netherlands</option>
# <option value="240">New Caledonia</option>
# <option value="67">New Zealand</option>
# <option value="124">Nicaragua</option>
# <option value="105">Niger</option>
# <option value="106">Nigeria</option>
# <option value="150">North Korea</option>
# <option value="191">Northern Ireland</option>
# <option value="180">Northern Mariana Islands</option>
# <option value="68">Norway</option>
# <option value="151">Oman</option>
# <option value="89">Other</option>
# <option value="152">Pakistan</option>
# <option value="181">Palau</option>
# <option value="204">Palestinian Territories</option>
# <option value="70">Panama</option>
# <option value="182">Papua New Guinea</option>
# <option value="125">Paraguay</option>
# <option value="94">Peru</option>
# <option value="71">Philippines</option>
# <option value="72">Poland</option>
# <option value="73">Portugal</option>
# <option value="200">Principality of Monaco</option>
#  <option value="126">Puerto Rico</option>
# <option value="249">Republic of Congo</option>
# <option value="169">Republic of Macedonia</option>
# <option value="173">Romania</option>
# <option value="74">Russia</option>
# <option value="107">Rwanda</option>
# <option value="247">Saint Kitts and Nevis</option>
# <option value="183">Samoa</option>
# <option value="234">San Marino</option>
# <option value="235">São Tomé and Príncipe</option>
# <option value="153">Saudi Arabia</option>
# <option value="190">Scotland</option>
# <option value="212">Senegal</option>
# <option value="76">Serbia</option>
# <option value="241">Seychelles</option>
# <option value="77">Sierra Leone</option>
# <option value="90">Singapore</option>
# <option value="253">Sint Maarten</option>
# <option value="78">Slovakia</option>
# <option value="95">Slovenia</option>
# <option value="184">Solomon Islands</option>
# <option value="108">South Africa</option>
# <option value="154">South Korea</option>
# <option value="79">Spain</option>
# <option value="155">Sri Lanka</option>
# <option value="199">St. Lucia</option>
# <option value="197">St. Vincent &amp; the Grenadines</option>
# <option value="110">Sudan</option>
# <option value="203">Surinam</option>
# <option value="109">Swaziland</option>
# <option value="80">Sweden</option>
# <option value="81">Switzerland</option>
# <option value="225">Syria</option>
# <option value="82">Taiwan</option>
# <option value="156">Tajikistan</option>
# <option value="207">Tanzania</option>
# <option value="83">Thailand</option>
# <option value="84">Togo</option>
# <option value="186">Tokelau</option>
# <option value="185">Tonga</option>
# <option value="127">Trinidad and Tobago</option>
# <option value="112">Tunisia</option>
# <option value="157">Turkey</option>
# <option value="158">Turkmenistan</option>
# <option value="243">Turks and Caicos Islands</option>
# <option value="111">Uganda</option>
# <option value="175">Ukraine</option>
# <option value="159">United Arab Emirates</option>
# <option value="86">United States</option>
# <option value="128">United States Virgin Islands</option>
# <option value="129">Uruguay</option>
# <option value="194">Uzbekistan</option>
# <option value="187">Vanuatu</option>
# <option value="92">Venezuela</option>
# <option value="87">Vietnam</option>
# <option value="88">Wales</option>
# <option value="160">Yemen</option>
# <option value="113">Zambia</option>
# <option value="114">Zimbabwe</option>
# </select>'''
# ids_list = re.findall(r'\d{1,3}', list_)
# ids_list.pop(74) #Russia
# ids_list.pop(86) #USA
# len(ids_list)

In [311]:
breweries['brewery_name'][1199] #stopped on north riding brewery

'Kloosterbrouwerij Haagsche Broeder'

In [312]:
len(breweries['brewery_name'])

2079

In [320]:
# rating_urls = ['https://untappd.com/brewery/top_rated?country_id=' + str(x) for x in ids_list] # USA
# breweries = get_breweries_by_country(rating_urls)
# brewery_urls = ['https://untappd.com/AntiFactory/beer']

# TODO (запустить вечером):
beers = get_beers_by_brewery(breweries['brewery_url'][1800:]) #done

https://untappd.com/ExtraMileBrewing ...
https://untappd.com/friendscompanybrewing ...
https://untappd.com/BrewbrosMalmo ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/BeerMonkey_Brewery ...
https://untappd.com/FjaderholmarnasBryggeri ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/Bryggverket ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/MalmoBrygghus ...
ge

https://untappd.com/BurgdorferGasthausbrauerei ...
https://untappd.com/BrasserieVirage ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/w/altes-tramdepot/16862 ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/hoppypeople ...
https://untappd.com/LaNebuleuse ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f8

getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/w/taiwan-tsing-beer-co/4717 ...
https://untappd.com/w/tanzania-breweries/11276 ...
https://untappd.com/w/serengeti-breweries-ltd/11277 ...
https://untappd.com/OutlawBrewingloei ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/w/happy-new-beer/196015 ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/FullMoonBrewwork ...
ht

getting page number 3
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/kant_rivne ...
https://untappd.com/Kumpel ...
https://untappd.com/KrapkaComma ...
https://untappd.com/Kalambet ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/w/pivnaya-duma-pivna-duma/48231 ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/NdsZIP ...
getting page number 1
getting page number 2
gett

getting page number 7
getting page number 8
getting page number 9
getting page number 10
getting page number 11
getting page number 12
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/LittleCottageBrewery ...
getting page number 1
getting page number 2
getting page number 3
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/HouseofFermentology ...
https://untappd.com/AfterthoughtBrewing ...
getting page number 1
getting page number 2
getting page number 3
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/w/fl

getting page number 15
getting page number 16
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/BarrelCulture ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
getting page number 6
getting page number 7
getting page number 8
getting page number 9
getting page number 10
getting page number 11
getting page number 12
getting page number 13
getting page number 14
getting page number 15
getting page number 16
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/MAZURTBrewingCompany ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver i

getting page number 3
getting page number 4
getting page number 5
getting page number 6
getting page number 7
getting page number 8
getting page number 9
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/PasteurStreetBrewingCompany ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
getting page number 6
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/7Bridges ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://u

Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/BigHandBeer ...
https://untappd.com/TwtLol ...
https://untappd.com/CaffleBrewery ...
https://untappd.com/VOG_BREWERY ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/MumblesBrewery ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

https://untappd.com/KingstoneBrewery ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.102

In [321]:
# добавить проверку на дубли
# было 45.6 Мб когда уходил
df = pd.DataFrame(beers)
# df.info()
for url, name, style, abv, ibu, rating, raters, added, brewery_url, brewery_added, description in tqdm(df.values):
    beers_table.insert({'beer_url': url, 'name': name, 'style': style, 'abv': abv, 'ibu': ibu,
                        'rating': rating, 'raters': raters, 'added': added, 'brewery_url': brewery_url,
                        'brewery_added': brewery_added, 'description': description})

# Дообогащаю БД с позициями пива

In [339]:
# функция возвращает перечень позиций пива и их х-к, указанных пивоварней
def update_beers_by_brewery(brewery_urls):
    d = {'url': [], 'name': [], 'style': [], 'abv': [], 'ibu': [], 'rating': [],
         'raters': [], 'added': [], 'brewery_url': [], 'brewery_added': [], 'description': []}
    
    options = webdriver.ChromeOptions()
    options.binary_location = "/Applications/Google Chrome 2.app/Contents/MacOS/Google Chrome"
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(brewery_urls[0] + '/beer')
    html = driver.page_source.encode('utf-8')
    # Click button to login screen
    time.sleep(7)
    driver.find_element_by_css_selector('.beer-list .button').click()
    # Login
    driver.find_element_by_id('username').send_keys('mershov')
    driver.find_element_by_id('password').send_keys('Aa1234567')
    driver.find_element_by_css_selector('span.button.yellow.submit-btn').click()
    
    for url in tqdm(brewery_urls):
        print(url, '...')
        url = url + '/beer'
        page_num = 0
        driver.get(url)
        
        # Click button 'Show more' while not error
        while driver.find_elements_by_css_selector('.beer-list .button'):#.beer-list .button
            try:
                driver.find_element_by_css_selector('.beer-list .button').click()#.beer-list .button
            except Exception as e:
                print(e)
                break
            page_num += 1
            print("getting page number "+str(page_num))
            if(page_num > 10):
                time.sleep(np.random.randint(25, 35))
            else:
                time.sleep(np.random.randint(7, 13))
                
        html = driver.page_source.encode('utf-8')
        
        names = Selector(text=html).xpath("//*[contains(@class, 'name')]/a/text()").extract()
        
        brewery_urls = [url] * len(names)
        
        brewery_added = Selector(text=html).xpath("//*[contains(@class, 'details brewery claimed') or contains(@class, 'details brewery')]//*[contains(@class, 'date')]/text()").extract()
        brewery_added = list(map(lambda s: re.findall(r'(\d{2}\/\d{2}\/\d{2})', s)[0], brewery_added))
        brewery_added = brewery_added * len(names)        
        
        urls = Selector(text=html).xpath("//*[contains(@class, 'name')]/a/@href").extract()
        
        styles = Selector(text=html).xpath("//*[contains(@class, 'beer-details')]//p[2]/text()").extract()
        
        abvs = Selector(text=html).xpath("//*[contains(@class, 'abv')]/text()").extract()
        abvs = list(map(lambda s: re.findall(r'\d{1,2}[.]?[\d{1,2}]?|N/A', s)[0], abvs))
        
        ibus = Selector(text=html).xpath("//*[contains(@class, 'ibu')]/text()").extract()
        ibus = list(map(lambda s: re.findall(r'(\d{1,3}|N/A) IBU', s)[0], ibus))
        
        ratings = Selector(text=html).xpath("//*[contains(@class, 'beer-item')]//*[contains(@class, 'num')]").extract()
        ratings = list(map(remove_tags, ratings))
        ratings = list(map(lambda s: re.findall(r'(\d{1}[\,\.]{0,1}\d{0,2}|N/A)', s)[0], ratings))
        
        raters = Selector(text=html).xpath("//*[contains(@class, 'beer-item')]//*[contains(@class, 'raters')]/text()").extract()
        raters = list(map(lambda s: re.findall(r'(\d{1,3}[\,\.]{0,1}\d{0,3}) Rating', s)[0], raters))
        raters = list(map(lambda s: s.replace(',', ''), raters))
        
        addeds = Selector(text=html).xpath("//*[contains(@class, 'beer-item')]//*[contains(@class, 'date')]/text()").extract()
        addeds = list(map(lambda s: re.findall(r'(\d{2}\/\d{2}\/\d{2})', s)[0], addeds))
        
        descriptions = Selector(text=html).xpath("//*[contains(@class, 'beer-details')]//p[4]").extract()
        descriptions = list(map(remove_tags, descriptions))
        descriptions = list(map(lambda s: s.replace(' Read Less', ''), descriptions))
        
        try:
            assert (len(names) == len(urls) ==\
                    len(brewery_urls) == len(descriptions) ==\
                    len(styles) == len(abvs) == len(ibus) ==\
                    len(raters) == len(ratings) == len(addeds) ==\
                    len(brewery_added))
        except:
            print('Different rows count for brewery:', url)
            continue
        d['name'] = names
        d['brewery_url'] = brewery_urls
        d['brewery_added'] = brewery_added
        d['url'] = ['https://untappd.com' + l for l in urls]
        d['style'] = styles
        d['abv'] = abvs
        d['ibu'] = ibus
        d['rating'] = ratings
        d['raters'] = raters
        d['added'] = addeds
        d['description'] = descriptions
        
        df = pd.DataFrame(d)
        try:
            total_beers = Selector(text=html).xpath("//*[contains(@class, 'details brewery claimed') or contains(@class, 'details brewery')]//*[contains(@class, 'count')]/text()").extract()
        except:
            total_beers = ['\nN/A']
        print('got information about', df.shape[0], 'of', total_beers[0][1:])
        
        i = 0
        for url, name, style, abv, ibu, rating, raters, added, brewery_url, brewery_added, description in df.values:
            if not beers_table.contains(Query().beer_url == url):
                i += 1
                beers_table.insert({'beer_url': url, 'name': name, 'style': style, 'abv': abv, 'ibu': ibu,
                                    'rating': rating, 'raters': raters, 'added': added, 'brewery_url': brewery_url,
                                    'brewery_added': brewery_added, 'description': description})
            else:
                continue
        print('updated', i, 'beers')
        
        time.sleep(np.random.randint(5, 10))
#     return d

In [348]:
breweries['brewery_url'][530]

'https://untappd.com/MillsBrewing'

In [352]:
%%time
update_beers_by_brewery(breweries['brewery_url'][1000:1300]) # done

https://untappd.com/BirrificiodeiCastelli ...
got information about 18 of 18 Beers 
updated 0 beers
https://untappd.com/w/birrificio-oldo/51663 ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 43 of 43 Beers 
updated 0 beers
https://untappd.com/w/piccolo-birrificio/4452 ...
got information about 15 of 15 Beers 
updated 0 beers
https://untappd.com/w/birrificio-artigianale-lariano/18682 ...
getting page number 1
getting page number 2
getting page number 3
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 73 of 73 Beers 
updated 0 beers
https://untappd.com/w/dada/44858 ...
getting page number 1
getting page number 2
ge

updated 1 beers
https://untappd.com/KyotoBrewing ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 77 of 77 Beers 
updated 1 beers
https://untappd.com/w/barbaric-works/302667 ...
getting page number 1
getting page number 2
getting page number 3
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 69 of 69 Beers 
updated 0 beers
https://untappd.com/w/yyg-brewery/260797 ...
getting page number 1
getting page number 2
getting page number 3
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c7244

updated 1 beers
https://untappd.com/w/aqula-brauhaus/14238 ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 109 of 109 Beers 
updated 0 beers
https://untappd.com/w/strange-brewing-lab/137374 ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 95 of 95 Beers 
updated 1 beers
https://untappd.com/w/yokohama-bay-brewing-co/27911 ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
Message: element not inter

updated 0 beers
https://untappd.com/Alkimikis ...
got information about 22 of 22 Beers 
updated 0 beers
https://untappd.com/Valmiermuiza ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 112 of 112 Beers 
updated 0 beers
https://untappd.com/LatgalesAlus ...
got information about 18 of 18 Beers 
updated 0 beers
https://untappd.com/MadonasAlus ...
got information about 8 of 8 Beers 
updated 0 beers
https://untappd.com/w/sia-abula/31391 ...
got information about 5 of 5 Beers 
updated 0 beers
https://untappd.com/w/aldaris/19 ...
getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Drive

getting page number 5
getting page number 6
getting page number 7
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 161 of 161 Beers 
updated 111 beers
https://untappd.com/w/aukstaitijos-bravorai/70219 ...
getting page number 1
getting page number 2
getting page number 3
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 56 of 56 Beers 
updated 6 beers
https://untappd.com/w/kauen-craft/266725 ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 48 of 48 Beers 

getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 27 of 27 Beers 
updated 0 beers
https://untappd.com/w/monzon-brewing-co/355750 ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 30 of 30 Beers 
updated 5 beers
https://untappd.com/w/cerveceria-ramuri/3257 ...
getting page number 1
getting page number 2
getting page number 3
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 63 of 63 Beers 
updated 38 beers
https:/

got information about 9 of 9 Beers 
updated 0 beers
https://untappd.com/w/myanmar-brewery-limited/2277 ...
got information about 8 of 8 Beers 
updated 0 beers
https://untappd.com/w/mandalay-brewery/43457 ...
got information about 5 of 5 Beers 
updated 0 beers
https://untappd.com/w/dagon-beverages-company-limited/36543 ...
got information about 9 of 9 Beers 
updated 0 beers
https://untappd.com/w/camelthorn-breweries/9581 ...
got information about 12 of 12 Beers 
updated 0 beers
https://untappd.com/NamibiaBreweries ...
got information about 15 of 15 Beers 
updated 0 beers
https://untappd.com/SherpaBrewery ...
got information about 6 of 6 Beers 
updated 0 beers
https://untappd.com/w/gorkha-brewery/3368 ...
got information about 5 of 5 Beers 
updated 0 beers
https://untappd.com/CGBrewery ...
got information about 9 of 9 Beers 
updated 0 beers
https://untappd.com/w/mount-everest-brewery/5946 ...
got information about 5 of 5 Beers 
updated 0 beers
https://untappd.com/TSwildales ...
getting p

updated 1 beers
https://untappd.com/BigBellyBrewingNL ...
got information about 16 of 16 Beers 
updated 0 beers
https://untappd.com/NeptunusBier ...
got information about 19 of 19 Beers 
updated 0 beers
https://untappd.com/BrouwgildeAbdijRolduc ...
got information about 14 of 14 Beers 
updated 0 beers
https://untappd.com/Oproer ...
got information about 15 of 15 Beers 
updated 0 beers
https://untappd.com/DeLastigeBroertjes ...
got information about 12 of 12 Beers 
updated 0 beers
https://untappd.com/w/zeeburg/17272 ...
got information about 7 of 7 Beers 
updated 0 beers
https://untappd.com/LOCBrewery ...
got information about 22 of 22 Beers 
updated 1 beers
https://untappd.com/CransCraftBeer ...
got information about 9 of 9 Beers 
updated 0 beers
https://untappd.com/gruttepierbrouwerij ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5

Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 55 of 55 Beers 
updated 0 beers
https://untappd.com/ChurBrewingCompany ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 27 of 27 Beers 
updated 0 beers
https://untappd.com/beerbaroness ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 41 of 41 Beers 
updated 0 beers
https://untappd.com/Weezledog ...
got information about 15 of 15 Beers 
updated 0 beers
https:

got information about 16 of 16 Beers 
updated 0 beers
https://untappd.com/Lacada ...
getting page number 1
getting page number 2
Message: element not interactable
  (Session info: chrome=70.0.3538.110)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.13.6 x86_64)

got information about 39 of 39 Beers 
updated 0 beers
CPU times: user 4h 2min 2s, sys: 36min 23s, total: 4h 38min 25s
Wall time: 8h 5min 27s


In [350]:
breweries['brewery_url'][533]

'https://untappd.com/w/old-chimneys-brewery/2563'

# TinyDB

In [ ]:
# Сейчас в db записано [0: 326), кроме России(74) и США(86)

In [322]:
db.tables

<bound method TinyDB.tables of <TinyDB tables=['_default', 'BREWERIES', 'BEERS'], tables_count=3, default_table_documents_count=0, all_tables_documents_count=['_default=0', 'BREWERIES=2079', 'BEERS=95723']>>

In [327]:
beers_table.search(Query().beer_url == 'https://untappd.com/b/juguetes-perdidos-jamaica-dubbel/1649744')

[{'beer_url': 'https://untappd.com/b/juguetes-perdidos-jamaica-dubbel/1649744',
  'name': 'Jamaica Dubbel',
  'style': 'Belgian Dubbel',
  'abv': '7',
  'ibu': '19',
  'rating': '4.13',
  'raters': '109',
  'added': '07/17/16',
  'brewery_url': 'https://untappd.com/JuguetesPerdidos/beer',
  'brewery_added': '09/06/15',
  'description': 'Clásica Dubbel de Abadía con adición de granos de Pimienta de Jamaica y otras especias.'}]

In [326]:
beers_table.contains(Query().name == 'Jamaica Dubbel')

True

In [173]:
# breweries_table = db.table('BREWERIES')

In [178]:
# beers_table = db.table('BEERS')

In [ ]:
# TODO: сделать функцию, вытаскивающую подробную информацию о пиве с комментариями

# DEBUG

In [54]:
url = 'https://untappd.com/w/medovarnya-tradicii-predkov/239419/beer'
options = webdriver.ChromeOptions()
options.binary_location = "/Applications/Google Chrome 2.app/Contents/MacOS/Google Chrome"
driver = webdriver.Chrome(chrome_options=options)
driver.get(url)
html = driver.page_source.encode('utf-8')
page_num = 0

# Click button to login screen
driver.find_element_by_css_selector('.beer-list .button').click()
# Login
driver.find_element_by_id('username').send_keys('mershov')
driver.find_element_by_id('password').send_keys('Aa1234567')
driver.find_element_by_css_selector('span.button.yellow.submit-btn').click()

# Click button while not error
while driver.find_elements_by_css_selector('.beer-list .button'):
    try:
        driver.find_element_by_css_selector('.beer-list .button').click()
    except:
        break
    page_num += 1
    print("getting page number "+str(page_num))
    time.sleep(1)

html = driver.page_source.encode('utf-8')

In [57]:
name = Selector(text=html).xpath("//*[contains(@class, 'name')]/a/text()").extract()

urls = Selector(text=html).xpath("//*[contains(@class, 'name')]/a/@href").extract()

descriptions = Selector(text=html).xpath("//*[contains(@class, 'beer-details')]//p[4]").extract()
descriptions = list(map(remove_tags, descriptions))
descriptions = list(map(lambda s: s.replace(' Read Less', ''), descriptions))

styles = Selector(text=html).xpath("//*[contains(@class, 'beer-details')]//p[2]/text()").extract()

abvs = Selector(text=html).xpath("//*[contains(@class, 'abv')]/text()").extract()
abvs = list(map(lambda s: re.findall(r'\d{1,2}[.]?[\d{1,2}]?|N/A', s)[0], abvs))

ibus = Selector(text=html).xpath("//*[contains(@class, 'ibu')]/text()").extract()
ibus = list(map(lambda s: re.findall(r'(\d{1,3}|N/A) IBU', s)[0], ibus))

raters = Selector(text=html).xpath("//*[contains(@class, 'beer-item')]//*[contains(@class, 'raters')]/text()").extract()
raters = list(map(lambda s: re.findall(r'(\d{1,3}[\,\.]{0,1}\d{0,3}) Rating', s)[0], raters))
raters = list(map(lambda s: s.replace(',', ''), raters))

ratings = Selector(text=html).xpath("//*[contains(@class, 'beer-item')]//*[contains(@class, 'num')]").extract()
ratings = list(map(remove_tags, ratings))
ratings = list(map(lambda s: re.findall(r'(\d{1}[\,\.]{0,1}\d{0,2}|N/A)', s)[0], ratings))

addeds = Selector(text=html).xpath("//*[contains(@class, 'beer-item')]//*[contains(@class, 'date')]/text()").extract()
addeds = list(map(lambda s: re.findall(r'(\d{2}\/\d{2}\/\d{2})', s)[0], addeds))

# <div class="details brewery claimed">
brewery_added = Selector(text=html).xpath("//*[contains(@class, 'details brewery claimed') or contains(@class, 'details brewery')]//*[contains(@class, 'date')]/text()").extract()
brewery_added = list(map(lambda s: re.findall(r'(\d{2}\/\d{2}\/\d{2})', s)[0], brewery_added))
brewery_added = brewery_added * len(name)

len(name), len(urls), len(descriptions), len(styles), len(abvs), len(ibus), len(raters), len(addeds), len(brewery_added), len(ratings)

(8, 8, 8, 8, 8, 8, 8, 8, 8, 8)

In [58]:
brewery_added

['12/05/15',
 '12/05/15',
 '12/05/15',
 '12/05/15',
 '12/05/15',
 '12/05/15',
 '12/05/15',
 '12/05/15']

In [116]:
# # append
# # with open('breweries.json', 'a') as fp:
# #     json.dump(breweries, fp)
# with open('beers2-2.json', 'a') as fp:
#     json.dump(beers, fp)

In [117]:
# len(beers['brewery_url'])

3072

In [25]:
url = 'https://untappd.com/brewery/top_rated?country_id=74'

class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

opener = AppURLopener()
response = opener.open(url)
html = response.file.read()

link = Selector(text=html).xpath("//*[contains(@class, 'name')]/a/@href").extract()
country = Selector(text=html).xpath("//*[contains(@class, 'style')]/text()").extract()
name = Selector(text=html).xpath("//*[contains(@class, 'name')]/a/text()").extract()
len(link), len(country), len(name)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  


(50, 50, 50)